In [1]:
import ray
ray.init(num_cpus = 4)

2020-09-20 21:29:38,485	INFO resource_spec.py:223 -- Starting Ray with 3.12 GiB memory available for workers and up to 1.57 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-09-20 21:29:38,991	INFO services.py:1191 -- View the Ray dashboard at localhost:8265


{'node_ip_address': '192.168.1.59',
 'raylet_ip_address': '192.168.1.59',
 'redis_address': '192.168.1.59:6379',
 'object_store_address': '/tmp/ray/session_2020-09-20_21-29-38_484239_7684/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2020-09-20_21-29-38_484239_7684/sockets/raylet',
 'webui_url': 'localhost:8265',
 'session_dir': '/tmp/ray/session_2020-09-20_21-29-38_484239_7684'}

In [40]:
import numpy as np, matplotlib.pyplot as plt, sklearn, scipy as sp
import sys; sys.path.insert(0, '../')
from Toolbox import infcy
def log(x):
    return np.ma.filled(np.log(np.ma.masked_equal(x, 0)), 0)

from sklearn.neighbors import NearestNeighbors
# @ray.remote
def f(tmp):
    
    ni, x, pNN, k = tmp
    # assume gaussian on point, estimate the hyper-parameters
    k_distance = pNN[0][ni]

    b  = k_distance[k]
    a_ = x[ni] - b
    b_ = x[ni] + b


    mu_idx = pNN[1][ni]
    mu     = x[mu_idx].mean(0)
    sigma = np.cov(x[mu_idx].T)
    gauss = sp.stats.multivariate_normal(mu, sigma,\
                                        allow_singular = 1)
    gxi = gauss.pdf(x[ni])

    Gi   = gauss.cdf(b_) - gauss.cdf(a_)
    H_c = 1/n * (log(Gi) - log(gxi))
    return H_c
def kpn_entropy(x, k, p):
    assert p >= k
    if x.ndim < 2:
        x = x[:, None]
    n = x.shape[0]
    pNN = NearestNeighbors(n_neighbors = p, metric = 'chebyshev').fit(x).kneighbors()
    H = sp.special.digamma(n) - sp.special.digamma(k)
    H_c = 0
    
    import multiprocessing as mp
    print()
    with mp.Pool() as p:
        h = sum(p.map(f, ((i, x, pNN, k) for i in range(n))))
#     print(h, H)
    return H + h
# np.random.seed(0)
n = int(1e4)
m = 1
means = np.zeros(m)
cov   = np.eye(means.size) # * .5 + .5
dist = sp.stats.multivariate_normal(means, cov)
x = dist.rvs(n)
# print(x.shape); assert 0
x = np.random.choice([-1, 1], size = (n, 1)) + np.random.randn(n, 1) * .2
theta = .04
p  = int(theta * n)
k  = 2
h = kpn_entropy(x, k, int(theta * n)) 

# h = infcy.kpn_entropy(x, k, p)
true = 1/2 * np.log(np.linalg.det(2 * np.pi * np.exp(1) * dist.cov))
# print(h, true)
# print((h - true)/true, h , true, 1)
print(h)
# print(h, true, (h-true)/true * 100)


1.0056358635291085


In [4]:
%%timeit 
hpy = kpn_entropy(x, k, p)

TypeError: 'int' object is not subscriptable

In [5]:
%%timeit
hcy = infcy.kpn_entropy(x[:, None], k, p)

ValueError: Buffer has wrong number of dimensions (expected 2, got 3)

In [4]:
print(hpy, hcy)

NameError: name 'hpy' is not defined